This notebook is for the analysis of experiments in which some flies were interacting with balls that were blocked using magnets. After a certain amount of time, magnets were removed and we want to compare how well they push balls compared to controls.

# Required packages 


In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np

from scipy import stats
from statsmodels.stats.multitest import multipletests

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import iqplot

import importlib

import holoviews as hv
import bokeh.io
import colorcet

bokeh.io.output_notebook()

hv.extension("bokeh")

In [ ]:
# Select the experiments to analyze

Let's start with one that is representative of the best settings I had, which was using magnets under the backlighting sheet, keeping control flies in their chambers until the first half-hour of pretraining was finished.
These were done on **240530 and 240531**

In [ ]:
# Get the data path
Datapath = Utils.get_data_path()

# Get all folders with "TNT_Fine" in the name

Folders = [
    f
    for f in os.listdir(Datapath)
    if "240530" in f or "240531" in f and os.path.isdir(Datapath / f)
]

Folders

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
# Generate Experiment objects from each folder

Experiments = [Ballpushing_utils.Experiment(Datapath / f) for f in Folders]

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
# Build the position

magnet_data = Ballpushing_utils.Dataset(Experiments)

print(magnet_data)

In [ ]:
start = 30 * 60

magnet_data.generate_dataset("summary", time_range=[start])

In [ ]:
# Get sample size : get how many unique fly ids we have grouped by Magnet

magnet_data.data.groupby("Magnet")["fly"].nunique()

# Create a "label" column that is "Magnet block" if Magnet is y, and "Control" if Magnet is n. Add the sample size to the label.

magnet_data.data["label"] = magnet_data.data["Magnet"].apply(
    lambda x: "Magnet block (n = 59)" if x == "y" else "Control (n = 48)"
)


In [ ]:
# Get the data columns

data_columns = magnet_data.data.columns

data_columns

# Do some plotting

Now we're gonna plot some metrics and compare them between Magnet y and magnet n.

In [ ]:
importlib.reload(HoloviewsTemplates)

In [ ]:
Plot = HoloviewsTemplates.jitter_boxplot(
    data=magnet_data.data,
    metric ="TimeToFinish",
    kdims = "label",
    metadata=magnet_data.metadata,
    plot_options=HoloviewsTemplates.hv_slides,
    colorby="Genotype",
)

In [ ]:
Plot = Plot.opts(
    ylabel = "Time to bring the ball to the end (s)",
    ylim = (0, 3800)
)

In [ ]:
Plot
# + Histogram distribution

## Make and save all metrics plots

In [ ]:
savepath = Utils.get_labserver() / "Experimental_data/MultiMazeRecorder/Plots/MagnetBlock/240617"

In [ ]:
hv.save(Plot, savepath / "TimeToFinish_byGenotype.html")

In [ ]:
# Also get the png
hv.save(Plot, savepath / "TimeToFinish_byGenotype.png", fmt="png")

In [ ]:
# Do the same with NumberEvents

Plot = HoloviewsTemplates.jitter_boxplot(
    data=magnet_data.data,
    metric="NumberEvents",
    kdims="label",
    metadata=magnet_data.metadata,
    plot_options=HoloviewsTemplates.hv_slides,
    colorby="Genotype",
)

Plot = Plot.opts(
    ylabel = "Number of events",
)

hv.save(Plot, savepath / "NumberEvents_byGenotype.html")

In [ ]:
# Generate the list of metrics

metrics = [
    "NumberEvents",
    "FinalEvent",
    "FinalTime",
    "SignificantEvents",
    "SignificantFirst",
    "SignificantFirstTime",
    "Pushes",
    "Pulls",
    "PullingRatio",
    "InteractionProportion",
    "AhaMoment",
    "AhaMomentIndex",
    "InsightEffect",
    "TimeToFinish",
    "SignificantRatio",
]

In [ ]:
# For each metric, generate a jitter boxplot and save it

for metric in metrics: 
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data, metric, "Magnet", magnet_data.metadata, scale_max=[True if metric == "FinalTime" else False]
    )
    hv.save(Plot, savepath / f"{metric}.html")

In [ ]:
# Make a nice layout with all the plots:
plots = []

for metric in metrics:
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data,
        metric,
        "Magnet",
        magnet_data.metadata,
        scale_max=[True if metric in ["FinalTime", "SignificantRatio"] else False],
    )
    plots.append(Plot)

layout = hv.Layout(plots).cols(2)

In [ ]:
hv.save(layout, savepath / "All_metrics.html")

In [ ]:
importlib.reload(HoloviewsTemplates)

# Distribution

We can see that there seems to be some distribution variability between groups, which would be better highlighted by using something like an histogram

In [ ]:
Hist = HoloviewsTemplates.histograms(magnet_data.data, "TimeToFinish", "label", bins = 20, xlabel = "Time to bring the ball to the end (s)", plot_options=HoloviewsTemplates.hv_slides, orientation = "horizontal")

In [ ]:
Hist

In [ ]:
hv.save(Hist, savepath / "TimeToFinish_histogram.png")

# Heatmaps

Let's look at the ball position over time grouped by Magnet condition

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
start = 30 * 60

In [ ]:
position = magnet_data.generate_dataset(metrics = "coordinates", time_range=[start])

## Sanity checks

> The dataset should be 3600 seconds long. So 3600 * 29 rows.

In [ ]:
position.head()

In [ ]:
# Get the data columns

data_columns = position.columns

data_columns

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.plotting.util import process_cmap
from bokeh.palettes import Viridis256
from bokeh.models import FixedTicker
import dask.dataframe as dd

# Convert the DataFrame to a Dask DataFrame
ddf = dd.from_pandas(position, npartitions=4)  # Use a suitable number of partitions

# Convert 'Magnet' and 'time' to categorical data type
ddf["Magnet"] = ddf["Magnet"].astype("category")
ddf["time"] = ddf["time"].astype("category")

# Ensure that the categories of 'time' are known
ddf["time"] = ddf["time"].cat.as_known()

# Define custom colormap
cmap = process_cmap("Viridis256_r", provider="bokeh")

# Get unique genotypes
Genotypes = ddf["Magnet"].unique().compute()


# Define a function that removes y-axis ticks
def remove_yticks(plot, element):
    plot.handles["yaxis"].ticker = FixedTicker(ticks=[])


# Pivot the DataFrame once
df_pivot = ddf.categorize(["Magnet", "time"]).pivot_table(
    index="fly", columns="time", values="yball_relative", aggfunc="first"
)

for genotype in Genotypes:
    # Slice the pivoted DataFrame for the current genotype
    df_genotype = df_pivot.loc[ddf[ddf["Magnet"] == genotype]["fly"].unique()]

    # Create a HeatMap
    heatmap = hv.HeatMap((df_genotype.columns, df_genotype.index, df_genotype.values))

    # Apply options to the HeatMap
    heatmap.opts(
        opts.HeatMap(
            cmap=cmap,
            colorbar=True,
            tools=["hover"],
            width=900,
            height=900,
            title=genotype,
            xlabel="Time(s)",
            ylabel="",
            fontscale=1.5,
        )
    )

    # Save each plot as a separate file in SVG format
    hv.save(heatmap, f"{savepath}/heatmap_{genotype}.svg", fmt="svg")

In [ ]:
Plotlist[0]

In [ ]:
# Create a layout with all the plots
layout = hv.Layout(Plotlist).cols(2)

layout

In [ ]:
# Save the layout
hv.save(layout, savepath / "Heatmaps.html")

# New protocol

Here I'm gonna replicate the analysis with a new protocol where control flies get to explore part of the corridor and have a transparent gate to see the ball and so the gate lifting doesn't provide a clear cue that they can push now.

In [ ]:
# Get the data path
Datapath = Utils.get_data_path()

# Get all folders with "TNT_Fine" in the name

Folders = [
    f
    for f in os.listdir(Datapath)
    if "240710" in f or "240711" in f and os.path.isdir(Datapath / f)
]

Folders

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:

# Generate Experiment objects from each folder

Experiments = [Ballpushing_utils.Experiment(Datapath / f) for f in Folders]

In [ ]:
# Build the position

magnet_data = Ballpushing_utils.Dataset(Experiments)

print(magnet_data)
start = 60 * 60

magnet_data.generate_dataset("summary", time_range=[start])

In [ ]:
preview = magnet_data.data
preview

In [ ]:
# Get the data columns

data_columns = magnet_data.data.columns

data_columns

In [ ]:
# Get sample size : get how many unique fly ids we have grouped by Magnet

magnet_data.data.groupby("Magnet")["fly"].nunique()

In [ ]:
# Create a "label" column that is "Magnet block" if Magnet is y, and "Control" if Magnet is n. Add the sample size to the label.

magnet_data.data["label"] = magnet_data.data["Magnet"].apply(
    lambda x: "Magnet block (n = 53)" if x == "y" else "Control (n = 54)"
)

## Event based metric jitterboxplots

In [ ]:
Plot = HoloviewsTemplates.jitter_boxplot(
    data=magnet_data.data,
    metric="TimeToFinish",
    kdims="label",
    metadata=magnet_data.metadata,
    plot_options=HoloviewsTemplates.hv_slides,
    colorby="Magnet",
)
Plot = Plot.opts(ylabel="Time to bring the ball to the end (s)", ylim=(0, 3800))


In [ ]:
savepath = Path("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/MagnetBlock/240718")

In [ ]:
hv.save(Plot, savepath / "TimeToFinish_byGenotype.html")

In [ ]:
# Generate the list of metrics

metrics = [
    "NumberEvents",
    "FinalEvent",
    "FinalTime",
    "SignificantEvents",
    "SignificantFirst",
    "SignificantFirstTime",
    "Pushes",
    "Pulls",
    "PullingRatio",
    "InteractionProportion",
    "AhaMoment",
    "AhaMomentIndex",
    "InsightEffect",
    "TimeToFinish",
    "SignificantRatio",
]
# For each metric, generate a jitter boxplot and save it

for metric in metrics:
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data,
        metric,
        "label",
        magnet_data.metadata,
        scale_max=[True if metric == "FinalTime" else False],
    )
    hv.save(Plot, savepath / f"{metric}.html")
    hv.save(Plot, savepath / f"{metric}.png", fmt="png")
# Make a nice layout with all the plots:
plots = []

for metric in metrics:
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data,
        metric,
        "label",
        magnet_data.metadata,
        scale_max=[True if metric in ["FinalTime", "SignificantRatio"] else False],
    )
    plots.append(Plot)

layout = hv.Layout(plots).cols(2)
hv.save(layout, savepath / "All_metrics.html")

# Also get the png
hv.save(layout, savepath / "All_metrics.png", fmt="png")

## Distribution

In [ ]:
Hist = HoloviewsTemplates.histograms(
    magnet_data.data,
    "TimeToFinish",
    "label",
    bins=20,
    xlabel="Time to bring the ball to the end (s)",
    plot_options=HoloviewsTemplates.hv_slides,
    orientation="horizontal",
)
Hist
hv.save(Hist, savepath / "TimeToFinish_histogram.png")